# Do not use - outdated! Use latest version of Rotator_interlock_test.ipynb

# Synchronous Interlock CCW/Rotator Integration Test

This notebook performs a synchronous motion interlock scenario integration test between the Camera Cable Wrap (CCW) and the Rotator with the Camera Cable Wrap tracking the Rotator. It includes enough boilerplate to allow the test to run at any time by getting current time information from the pointing and computing appropriate coordinates to slew.

In [120]:
import logging
import yaml

import os
import sqlite3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

In [121]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [122]:
def parallactic_angle(location, lst, target):
    """
    Calculate the parallactic angle.
    Parameters
    ----------
    time : `~astropy.time.Time`
        Observation time.
    target : `~astroplan.FixedTarget` or `~astropy.coordinates.SkyCoord` or list
        Target celestial object(s).
    grid_times_targets: bool
        If True, the target object will have extra dimensions packed onto the end,
        so that calculations with M targets and N times will return an (M, N)
        shaped result. Otherwise, we rely on broadcasting the shapes together
        using standard numpy rules.
    Returns
    -------
    `~astropy.coordinates.Angle`
        Parallactic angle.
    Notes
    -----
    The parallactic angle is the angle between the great circle that
    intersects a celestial object and the zenith, and the object's hour
    circle [1]_.
    .. [1] https://en.wikipedia.org/wiki/Parallactic_angle
    """
    # Eqn (14.1) of Meeus' Astronomical Algorithms
    H = (lst - target.ra).radian
    q = np.arctan2(np.sin(H),
                   (np.tan(location.lat.radian) *
                    np.cos(target.dec.radian) -
                    np.sin(target.dec.radian)*np.cos(H)))*u.rad
    return Angle(q)

In [123]:
test_message = "Interlock Rotator_CCW Integration Test"

In [124]:
d = salobj.Domain()

In [125]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "Rotator")
mtptg = salobj.Remote(d, "MTPtg")
mtm = salobj.Remote(d, "MTMount")

In [126]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task,
                     mtm.start_task)

RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages


[None, None, None, None]

RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 100 messages
RemoteEvent(MTMount, 0, mountWarning) falling behind; read 21 messages
RemoteEvent(MTMount, 0, mountWarning) falling behind; read 21 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteEvent(MTMount, 0, mountWarning) falling behind; read 21 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteEvent(MTMount, 0, mountWarning) falling behind; read 21 messages
RemoteEvent(Rotator, 0, summaryState) falling behind; read 63 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 32 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 47 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling

In [127]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

RemoteEvent(MTMount, 0, mountState) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTimesToLimits) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteEvent(MTMount, 0, mountError) falling behind; read 38 messages
RemoteEvent(MTMount, 0, mountError) falling behind; read 38 messages
RemoteEvent(MTMount, 0, mountError) falling behind; read 38 messages
RemoteEvent(Rotator, 0, tracking) falling behind; read 68 messages
RemoteEvent(MTMount, 0, mountError) falling behind; read 38 messages
RemoteEvent(Rotator, 0, interlock) falling behind; read 24 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTimesToLimits) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTimesToLimits) fallin

RuntimeError: Cannot get summaryState from MTPtg

In [129]:
await mtm.cmd_enterControl.start(timeout=10.)
await mtm.cmd_start.start(timeout=10.)
await mtm.cmd_enable.start(timeout=10.)

RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling beh

# Move to 0 deg Starting Position

In [ ]:
print("Move to 0.0 deg starting position")

alt = 45. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


# Calculating the other parameters     
rot_pa = para_ang-180*u.deg+rot_tel
print(rot_pa)
await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180+rot_pa.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=30)
await rot.cmd_stop.start(timeout=30)
await mtm.cmd_stop.start(timeout=30)

# Start Test

In [4]:
print("Test Start")
print("2019-12-12 05:06:02")
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message}")

Test Start
2019-12-12 05:06:02


# Set Low Velocity and Move Through Positive Interlock

In [132]:
await rot.cmd_configureVelocity.set_start(vlimit=0.5)

RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 18 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 18 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 18 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 18 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messag

In [133]:
await asyncio.sleep(1.)

print("Move to 10 deg position")

await rot.cmd_positionSet.set_start(angle=29.04,timeout=30.)

await rot.cmd_move.start(timeout=30.)

print("Reset the Rotator and CCW")

Move to 10 deg position


RemoteEvent(Rotator, 0, heartbeat) falling behind; read 33 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 33 messages


Reset the Rotator and CCW


Bring the Rotator and CCW back to enabled state

In [134]:
await mtm.cmd_clearerror.start(timeout=30)

RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0

Wait for Safety Reset

In [135]:
await rot.cmd_clearError.start(timeout=30)

RemoteEvent(MTMount, 0, mountState) falling behind; read 34 messages
RemoteEvent(MTMount, 0, mountState) falling behind; read 34 messages
RemoteEvent(MTMount, 0, mountState) falling behind; read 34 messages
RemoteEvent(MTMount, 0, mountState) falling behind; read 34 messages
RemoteEvent(MTMount, 0, mountState) falling behind; read 34 messages


In [136]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

[<State.OFFLINE: 4>,
 <State.STANDBY: 5>,
 <State.DISABLED: 1>,
 <State.ENABLED: 2>]

In [137]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

# Move to Zero

In [ ]:
print("Move to 0.0 deg starting position")

await rot.cmd_positionSet.set_start(angle=-0.5,timeout=30.)

await rot.cmd_move.start(timeout=30.)

Wait for override off

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

# Set Low Velocity and Move Through Negative Interlock

In [138]:
await asyncio.sleep(1.)

print("Move to -10 deg position")

await rot.cmd_positionSet.set_start(angle=9.04,timeout=30.)

await rot.cmd_move.start(timeout=30.)

print("Reset the Rotator and CCW")

RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling beh

Move to -10 deg position


RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 10 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling beh

Reset the Rotator and CCW


Bring the Rotator and CCW back to enabled state

In [ ]:
await mtm.cmd_clearerror.start(timeout=30)

Wait for override off

In [ ]:
await rot.cmd_clearError.start(timeout=30)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

# Move to Zero

In [ ]:
print("Move to 0.0 deg starting position")

await rot.cmd_positionSet.set_start(angle=0.0,timeout=30.)

await rot.cmd_move.start(timeout=30.)

# Test Complete

In [3]:
print("Test complete")
print("2019-12-12 05:12:02")
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"END - {test_message}")

Test complete
2019-12-12 05:12:02


# Bring the Rotator and CCW back to enabled state

In [ ]:
await mtm.cmd_clearerror.start(timeout=30)

Wait for override off

In [ ]:
await rot.cmd_clearError.start(timeout=30)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

# Move to sync position with CCW

In [130]:
print("Move to X.X deg starting position")

await rot.cmd_positionSet.set_start(angle=19.04,timeout=30.)

await rot.cmd_move.start(timeout=30.)

RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages


Move to X.X deg starting position


RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind;

# Additional Error Handling Commands

In [ ]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await rot.cmd_exitControl.start(timeout=10.)

In [ ]:
await rot.cmd_enterControl.start(timeout=10.)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await rot.cmd_stop.start(timeout=10.)

In [ ]:
await mtm.cmd_clearerror.start(timeout=10)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10)
await mtm.cmd_stop.start(timeout=10)

In [ ]:
await mtm.cmd_stop.start(timeout=10)

Bring CCW Down

In [ ]:
await mtm.cmd_disable.start(timeout=30.)
await mtm.cmd_standby.start(timeout=30.)
await mtm.cmd_exitControl.start(timeout=30.)

Wait for override off

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=30)